In [2]:
%%bash
pip install instructure-dap-client

StatementMeta(, e6a20fe5-a69e-43e8-aa38-8d4fc09f155b, 4, Finished, Available)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.5/69.5 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 76.4 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 43.7 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.3/140.3 kB 62.3 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.9
    Uninstalling SQLAlchemy-2.0.9:
      Successfully uninstalled SQLAlchemy-2.0.9
  Attempting uninstall: PyJWT
    Found existing installation: PyJWT 2.6.0
    Uninstalling PyJWT-2.6.0:
      Successfully uninstalled PyJWT-2.6.0


In [3]:
from datetime import datetime
import subprocess
import multiprocessing

def run_bash_command(command):
    subprocess.run(command, shell=True)

api_url = "https://api-gateway.instructure.com"
client_id = "us-west-2#8cb98924-70a9-49b1-bfe9-393e6ae111d5"
client_secret = "3zlv-vTSxyZvnR7grsv56PYBnAgy_N4qug1DP-YIzQs"

# define tables to be extracted from Canvas
tables = [
    "users"
]

commands = []

# create string timestamp to serve as directory
now = datetime.now()
date_ts = now.strftime("%Y%m%d%H%M%S")

# build bash command for the incremental load of each table
for table in tables:

    # retrieve most-recent load timestamp for 'since' parameter
    query  = f"SELECT MAX(ts) FROM {table}"
    result = spark.sql(query).collect()
    #since  = result[0][0]
    since = "2023-05-03T20:24:58"

    # build the command string
    file_path = f"/lakehouse/default/Files/Canvas/{table}/{date_ts}"
    commands.append(f"dap --base-url {api_url} --client-id {client_id} --client-secret {client_secret} incremental --format parquet --output-directory {file_path} --table {table} --namespace canvas --since {since}")


# create and start a separate process for each command
processes = [multiprocessing.Process(target=run_bash_command, args=(cmd,)) for cmd in commands]

for process in processes:
    process.start()

# wait for all processes to complete
for process in processes:
    process.join()

StatementMeta(, e6a20fe5-a69e-43e8-aa38-8d4fc09f155b, 5, Finished, Available)

2023-08-07 18:25:51,040 - INFO - Query started with job ID: 393238c4-58d5-4b6c-aebb-a10dd92a41f9
2023-08-07 18:25:51,040 - INFO - Query job still in status: waiting. Checking again in 5 seconds...
2023-08-07 18:25:56,594 - INFO - Query job still in status: running. Checking again in 5 seconds...
2023-08-07 18:26:01,958 - INFO - Query job still in status: running. Checking again in 5 seconds...
2023-08-07 18:26:07,840 - INFO - Query job still in status: running. Checking again in 5 seconds...
2023-08-07 18:26:13,378 - INFO - Query job still in status: running. Checking again in 5 seconds...
2023-08-07 18:26:18,864 - INFO - Query job still in status: running. Checking again in 5 seconds...
2023-08-07 18:26:24,851 - INFO - Query job still in status: running. Checking again in 5 seconds...
2023-08-07 18:26:30,398 - INFO - Query job still in status: running. Checking again in 5 seconds...
2023-08-07 18:26:35,859 - INFO - Query job still in status: running. Checking again in 5 seconds...
202

In [4]:
# perform a merge between each staging table and delta table
for table in tables:

    # transform the incremental data and store to a temporary table
    df = spark.read.parquet(f"Files/Canvas/{table}/{date_ts}/*/*.parquet")
    flat_df = df.select("key.*", "value.*", "meta.*")
    flat_df.createOrReplaceTempView(f"{table}_stage")

    updates = []
    inserts = []

    # dynamically build sql query components 
    for value in df.select("value.*").columns:
        
        updates.append(f"t.{value} = s.{value}")
        inserts.append(value)

    update  = "\n\t\t\t, ".join(updates)
    insert  = "\n\t\t\t, ".join(inserts)
    insert2 = "\n\t\t\t, s.".join(inserts)

    sql = f"""
        MERGE INTO {table} t
        USING {table}_stage s
        ON t.id = s.id
        WHEN MATCHED AND s.action = 'U' THEN 
        UPDATE SET
            {update}
            , t.ts = s.ts
        WHEN MATCHED AND s.action = 'D' THEN
        DELETE 
        WHEN NOT MATCHED AND s.action = 'I' THEN
        INSERT (
            id
            , {insert}
            , ts
        ) VALUES (
            s.id
            , {insert2}
            , s.ts
        )
    """

    # execute merge and show results
    display(spark.sql(sql))

StatementMeta(, e6a20fe5-a69e-43e8-aa38-8d4fc09f155b, 6, Finished, Available)

SynapseWidget(Synapse.DataFrame, b0640ca1-a9f9-423d-8f66-4e0243504673)